In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np
from datetime import datetime
from sklearn.metrics import pairwise

class DetectorDeRostos:
    def __init__(self, pasta_base_dados="rostos_detectados"):
        self.webcam = cv2.VideoCapture(0)  # para conectar o Python com a webcam
        self.reconhecimento_rosto = mp.solutions.face_detection  # ativando a solução de reconhecimento de rosto
        self.desenho = mp.solutions.drawing_utils  # ativando a solução de desenho
        self.reconhecedor_rosto = self.reconhecimento_rosto.FaceDetection(model_selection=0, min_detection_confidence=0.5)  # criando o item que consegue ler uma imagem e reconhecer os rostos ali dentro
        self.pasta_base_dados = pasta_base_dados

        # Criar pasta base de dados se não existir
        if not os.path.exists(self.pasta_base_dados):
            os.makedirs(self.pasta_base_dados)

        # Carregar rostos conhecidos
        self.rostos_conhecidos, self.nomes_rostos_conhecidos = self.carregar_rostos_conhecidos()

    def carregar_rostos_conhecidos(self):
        rostos_conhecidos = []
        nomes_rostos_conhecidos = []
        for nome_arquivo in os.listdir(self.pasta_base_dados):
            caminho_arquivo = os.path.join(self.pasta_base_dados, nome_arquivo)
            imagem = cv2.imread(caminho_arquivo)
            if imagem is not None:
                rostos_conhecidos.append(self.ajustar_tamanho_rosto(imagem))
                nomes_rostos_conhecidos.append(nome_arquivo)
        return rostos_conhecidos, nomes_rostos_conhecidos

    def ajustar_tamanho_rosto(self, rosto):
        rosto = cv2.cvtColor(rosto, cv2.COLOR_BGR2GRAY)
        rosto = cv2.resize(rosto, (150, 150))  # Redimensionar para uma dimensão fixa
        return rosto

    def salvar_rosto(self, imagem, bounding_box):
        # Extrair a área do rosto da imagem original
        altura, largura, _ = imagem.shape
        x_min = int(bounding_box.xmin * largura)
        y_min = int(bounding_box.ymin * altura)
        x_max = x_min + int(bounding_box.width * largura)
        y_max = y_min + int(bounding_box.height * altura)
        rosto = imagem[y_min:y_max, x_min:x_max]
        rosto_cinza = cv2.cvtColor(rosto, cv2.COLOR_BGR2GRAY)
        rosto_redimensionado = cv2.resize(rosto_cinza, (150, 150))  # Redimensionar para uma dimensão fixa

        # Criar um nome de arquivo único usando timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%f")
        caminho_arquivo = os.path.join(self.pasta_base_dados, f"rosto_{timestamp}.png")

        # Salvar o rosto
        cv2.imwrite(caminho_arquivo, rosto)
        print(f"Rosto salvo em: {caminho_arquivo}")

        # Atualizar a lista de rostos conhecidos
        self.rostos_conhecidos.append(self.ajustar_tamanho_rosto(rosto))
        self.nomes_rostos_conhecidos.append(caminho_arquivo)

    def reconhecer_rosto(self, rosto_desconhecido):
        rosto_desconhecido = self.ajustar_tamanho_rosto(rosto_desconhecido)
        rostos_conhecidos_array = np.array([rosto for rosto in self.rostos_conhecidos])

        if len(rostos_conhecidos_array) == 0:
            return False, None

        # Calcular a distância euclidiana entre o rosto desconhecido e todos os rostos conhecidos
        distancias = pairwise.euclidean_distances(rosto_desconhecido.flatten().reshape(1, -1), rostos_conhecidos_array.reshape(len(self.rostos_conhecidos), -1))

        # Verificar se a menor distância é abaixo do limiar
        limiar = 5000  # Ajustar conforme necessário
        indice_menor_distancia = np.argmin(distancias)
        menor_distancia = distancias[0][indice_menor_distancia]
        
        if menor_distancia < limiar:
            return True, self.nomes_rostos_conhecidos[indice_menor_distancia]
        else:
            return False, None

    def executar(self):
        while self.webcam.isOpened():
            validacao, frame = self.webcam.read()  # lê a imagem da webcam
            if not validacao:
                break
            imagem = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            lista_rostos = self.reconhecedor_rosto.process(imagem)  # usa o reconhecedor para criar uma lista com os rostos reconhecidos
            imagem = cv2.cvtColor(imagem, cv2.COLOR_RGB2BGR)

            if lista_rostos.detections:  # caso algum rosto tenha sido reconhecido
                for rosto in lista_rostos.detections:  # para cada rosto que foi reconhecido
                    bounding_box = rosto.location_data.relative_bounding_box
                    x_min = int(bounding_box.xmin * frame.shape[1])
                    y_min = int(bounding_box.ymin * frame.shape[0])
                    x_max = x_min + int(bounding_box.width * frame.shape[1])
                    y_max = y_min + int(bounding_box.height * frame.shape[0])

                    # Verificar se as coordenadas estão dentro dos limites da imagem
                    if x_min >= 0 and y_min >= 0 and x_max <= frame.shape[1] and y_max <= frame.shape[0]:
                        rosto_desconhecido = frame[y_min:y_max, x_min:x_max]

                        reconhecido, nome_arquivo = self.reconhecer_rosto(rosto_desconhecido)
                        if reconhecido:
                            cv2.putText(imagem, f"Reconhecido: {nome_arquivo}", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                            print(f"Rosto reconhecido: {nome_arquivo}")
                        else:
                            self.salvar_rosto(frame, bounding_box)
                            cv2.putText(imagem, "Novo rosto salvo", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                            print("Rosto novo salvo na base de dados.")
                    
                        self.desenho.draw_detection(imagem, rosto)  # desenha o rosto na imagem

            cv2.imshow("Rostos na sua webcam", imagem)  # mostra a imagem da webcam para a gente
            if cv2.waitKey(5) == 27:  # ESC # garante que o código vai ser pausado ao apertar ESC (código 27) e que o código vai esperar 5 milisegundos a cada leitura da webcam
                break
        self.webcam.release()  # encerra a conexão com a webcam
        cv2.destroyAllWindows()  # fecha a janela que mostra o que a webcam está vendo

if __name__ == "__main__":
    detector = DetectorDeRostos()
    detector.executar()